# Basic bond pricing

In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf
import plotly.graph_objects as go

## Bond pricing

### Method #1: Discount each cash flow

In [2]:
# Inputs
CR = 0.06           # annual coupon rate
YTM= 0.05           # annual yield to maturity
NOP= 2              # payments per year
TTM_PERIODS = 8     # time to maturity (in periods)
PRINCIPAL = 100     # face value of the bond

# Create dataframe to hold cash flows
bond = pd.DataFrame(dtype=float,columns=['Time (years)', 'Cash Flow', 'PV(Cash Flow)'], index=1+np.arange(TTM_PERIODS))
bond['Time (years)'] = bond.index / NOP
bond.index.name = 'Period'
bond

,Time (years),Cash Flow,PV(Cash Flow)
Period,,,
1,0.5,NaN,NaN
2,1.0,NaN,NaN
3,1.5,NaN,NaN
4,2.0,NaN,NaN
5,2.5,NaN,NaN
6,3.0,NaN,NaN
7,3.5,NaN,NaN
8,4.0,NaN,NaN


In [3]:
# Add the coupon cash flows
bond['Cash Flow'] = CR / NOP * PRINCIPAL
bond

,Time (years),Cash Flow,PV(Cash Flow)
Period,,,
1,0.5,3.0,NaN
2,1.0,3.0,NaN
3,1.5,3.0,NaN
4,2.0,3.0,NaN
5,2.5,3.0,NaN
6,3.0,3.0,NaN
7,3.5,3.0,NaN
8,4.0,3.0,NaN


In [4]:
# Add the face value cash flow
bond.loc[TTM_PERIODS,'Cash Flow'] = bond.loc[TTM_PERIODS,'Cash Flow'] + PRINCIPAL
bond

,Time (years),Cash Flow,PV(Cash Flow)
Period,,,
1,0.5,3.0,NaN
2,1.0,3.0,NaN
3,1.5,3.0,NaN
4,2.0,3.0,NaN
5,2.5,3.0,NaN
6,3.0,3.0,NaN
7,3.5,3.0,NaN
8,4.0,103.0,NaN


In [5]:
bond['PV(Cash Flow)'] = bond['Cash Flow'] / (1+YTM/NOP)**bond.index
bond


,Time (years),Cash Flow,PV(Cash Flow)
Period,,,
1,0.5,3.0,2.926829
2,1.0,3.0,2.855443
3,1.5,3.0,2.785798
4,2.0,3.0,2.717852
5,2.5,3.0,2.651563
6,3.0,3.0,2.586891
7,3.5,3.0,2.523796
8,4.0,103.0,84.536897


In [9]:
price = bond['PV(Cash Flow)'].sum()     # This is an aggregation command
print(f'Price is: ${price:,.2f}')

Price is: $103.59


Let's make the exercise above look a little nicer

In [10]:
# The following will format all cells with 2 decimal places, commas for thousands, and dollar signs
bond.style.format('${0:,.2f}')

# The following will format each column separately
format_dict = {'Time (years)':'{0:,.1f}', 
               'Cash Flow':'${0:,.2f}',
               'PV(Cash Flow)':'${0:,.2f}'}   # Other useful formats: '{:%m-%Y}' '{:.2%}'}
bond.style.format(format_dict)

,Time (years),Cash Flow,PV(Cash Flow)
Period,,,
1,0.5,$3.00,$2.93
2,1.0,$3.00,$2.86
3,1.5,$3.00,$2.79
4,2.0,$3.00,$2.72
5,2.5,$3.00,$2.65
6,3.0,$3.00,$2.59
7,3.5,$3.00,$2.52
8,4.0,$103.00,$84.54


In [11]:
# The formatting hasn't changed in the data; the above was just for display
bond

,Time (years),Cash Flow,PV(Cash Flow)
Period,,,
1,0.5,3.0,2.926829
2,1.0,3.0,2.855443
3,1.5,3.0,2.785798
4,2.0,3.0,2.717852
5,2.5,3.0,2.651563
6,3.0,3.0,2.586891
7,3.5,3.0,2.523796
8,4.0,103.0,84.536897


### Method #2: present value function

In [13]:
CR = 0.06           # annual coupon rate
YTM= 0.05           # annual yield to maturity
NOP= 2              # payments per year
TTM_PERIODS = 8     # time to maturity (in periods)
PRINCIPAL = 100     # face value of the bond

# Per-period discount rate
dr = YTM/NOP
# Per-period coupon
c = CR/NOP*PRINCIPAL

price = -npf.pv(dr,TTM_PERIODS,c,PRINCIPAL)
print(f'Price is: ${price:,.2f}')

Price is: $103.59


We can write a custom function to take specific inputs and give us a desired output.

In [14]:
def bondpv(cr,ytm,nop,ttm, principal):
    ''' 
    THIS IS A COMMENT SECTION
    WHEN WRITING FUNCTIONS, BEST PRACTICE IS TO DESCRIBE THE INPUTS AND OUTPUTS
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        p:              bond price
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    return p

In [15]:
# Now call the function
bondpv(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)

103.58506858373815

### Method #3: annuity formula

The present value of an annuity is:

$$ PV = \frac{C}{DR} \left( 1-\frac{1}{(1+DR)^T}\right) +  \frac{FACE}{(1+DR)^T} $$

In [18]:
dr = YTM/NOP
coupon = CR/NOP*PRINCIPAL
term1 = coupon/dr *(1-1/((1+dr)**TTM_PERIODS))   # what is term 1 conceptually?
term2 = PRINCIPAL/(1+dr)**TTM_PERIODS            # what is term 2 conceptually?
price = term1 + term2
print(f'Price is: ${price:,.2f}')

Price is: $103.59


We can convert this to a function as well.

In [20]:
def bondpv2(cr,ytm,nop,ttm, principal):
    ''' 
    THIS IS A COMMENT SECTION
    WHEN WRITING FUNCTIONS, BEST PRACTICE IS TO DESCRIBE THE INPUTS AND OUTPUTS
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        p:              bond price
    '''
    dr = ytm/nop
    c = cr/nop*principal
    pv_coupons = c/dr *(1-1/((1+dr)**ttm))
    pv_face    = principal/(1+dr)**ttm
    return pv_coupons + pv_face

In [21]:
# Now call the function
bondpv2(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)

103.58506858373815

## Fixed income prices and interest rates



In [67]:
# Calculate bond prices for a range of yields
yields = np.arange(0.005,0.125,0.005)
prices = [bondpv(CR,y,NOP,TTM_PERIODS,PRINCIPAL) for y in yields]

df = pd.DataFrame(prices,index=yields)
df.columns = ['price']
df.index.name='ytm'


In [68]:
# Plot the data
trace= go.Scatter(x=df.index, y=df.price, mode='lines')

fig = go.Figure()
fig.add_trace(trace)
fig.update_xaxes(title='Yield',tickformat=".2%")
fig.update_yaxes(title='Price',tickformat=".2f")
fig.update_layout(title='Bond Pricing')
fig.show()

View for different times to maturity

In [69]:
# Calculate bond prices for a range of yields

ttm_list = [8, 16, 40]
yields = np.arange(0.005,0.125,0.005)

df = pd.DataFrame(dtype=float,columns = ttm_list,index=yields)
df.index.name='ytm'
df.columns

Int64Index([8, 16, 40], dtype='int64')

In [70]:
for t in ttm_list:
    for y in yields:
        df.loc[y,t] = bondpv(CR,y,NOP,t,PRINCIPAL)
df

,8,16,40
ytm,,,
0.005,121.754548,143.078860,204.554624
0.010,119.557398,138.349813,190.430570
0.015,117.407380,133.804703,177.505611
0.020,115.303356,129.435748,165.669372
0.025,113.244218,125.235511,154.822133
0.030,111.228888,121.196896,144.873768
0.035,109.256316,117.313121,135.742787
0.040,107.325481,113.577709,127.355479
0.045,105.435388,109.984473,119.645142


In [71]:
# Plot the data
fig = go.Figure()
for t in ttm_list:
    ttm_years = int(t/NOP)
    trace= go.Scatter(x=df.index, y=df[t], mode='lines', name=str(ttm_years)+' years to maturity')
    fig.add_trace(trace)

fig.update_xaxes(title='Yield',tickformat=".2%")
fig.update_yaxes(title='Price',tickformat=".2f")
fig.update_layout(title='Bond Pricing')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.8))
fig.show()